In [1]:
# 使用0925_1训练数据训练得到学习器（SVM），结果可识别全部已测数据  注：不同模型结果都一样！
import pandas as pd
import numpy as np
import math
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import RandomizedLasso
from sklearn.feature_selection import RFE

In [87]:
train_data = pd.read_csv('train_data_1205.csv',header=0) #全部数据中的训练部分
test_data = pd.read_csv('test_data_1205_1.csv',header=0) #全部数据中的测试部分
X_train = train_data.drop('label',axis=1)
y_train = train_data['label']
X_test = test_data.drop('label',axis=1)
y_test = test_data['label']
print X_train.shape,y_train.shape,X_test.shape,y_test.shape

(120, 90) (120,) (125, 90) (125,)


In [4]:
def feature_selection(X_train,y_train,sorted_feature,selected_feature):
	score = 0
	while sorted_feature:
		selected_feature.append(sorted_feature[0])
		del sorted_feature[0]
		score_new,clf_best = model_selection(X_train,y_train,selected_feature)
		if score_new <= score:
			selected_feature.pop()
		else:
			score = score_new
			clf = clf_best
	return selected_feature,score,clf

In [82]:
def model_selection(X_train,y_train,feature):
	parameters={'C':[0.001,0.01,0.1,1,5,10,20]}
	lr = LogisticRegression()
	clf = GridSearchCV(lr,parameters,scoring='accuracy',cv=10)
	clf.fit(X_train[feature],y_train)
	return clf.best_score_,clf.best_estimator_

In [8]:
def feature_sort(X,y):
	# 卡方
	s1 = SelectKBest(chi2).fit(X,y).scores_
	s1[np.isnan(s1)] = 0
	s1=s1.reshape(-1,1)
	s1 = MinMaxScaler().fit_transform(s1)
	s1 = map(lambda x:round(x,4),s1)
	s1 = np.array(s1)
	# 皮尔森系数
	s2 = X.apply(lambda x:abs(pearsonr(x,y)[0])).values
	s2[np.isnan(s2)]=0
	s2=s2.reshape(-1,1)
	s2 = MinMaxScaler().fit_transform(s2)
	s2 = map(lambda x:round(x,4),s2)
	s2 = np.array(s2)
	# L2正则
	s3 = SelectFromModel(LogisticRegression(penalty="l2", C=1)).fit(X, y)
	l2 = s3.estimator_
	s3 = abs(l2.coef_[0])
	s3 = s3.reshape(-1,1)
	s3 = MinMaxScaler().fit_transform(s3)
	s3 = map(lambda x:round(x,4),s3)
	s3 = np.array(s3)
	# L1正则
	s4 = SelectFromModel(LogisticRegression(penalty="l1", C=1)).fit(X, y)
	l1 = s4.estimator_
	s4 = abs(l1.coef_[0])
	s4 = s4.reshape(-1,1)
	s4 = MinMaxScaler().fit_transform(s4)
	s4 = map(lambda x:round(x,4),s4)
	s4 = np.array(s4)
	# GBDT
	s5 =SelectFromModel(GradientBoostingClassifier()).fit(X, y)
	gb = s5.estimator_
	s5 = gb.feature_importances_
	s5 = s5.reshape(-1,1)
	s5 = MinMaxScaler().fit_transform(s5)
	s5 = map(lambda x:round(x,4),s5)
	s5 = np.array(s5)
	# 随机森林
	s6 = SelectFromModel(RandomForestRegressor()).fit(X, y)
	rf = s6.estimator_
	s6 = rf.feature_importances_
	s6 = s6.reshape(-1,1)
	s6 = MinMaxScaler().fit_transform(s6)
	s6 = map(lambda x:round(x,4),s6)
	s6 = np.array(s6)
	# 稳定性
	rlasso = RandomizedLasso()  # 默认通过AIC选择正则化系数alpha
	rlasso.fit(X, y)
	s7 = rlasso.scores_
	s7 = s7.reshape(-1,1)
	s7 = MinMaxScaler().fit_transform(s7)
	s7 = map(lambda x:round(x,4),s7)
	s7 = np.array(s7)
	# REF
	lr = LogisticRegression(penalty='l1')
	rfe = RFE(lr, n_features_to_select=1)
	rfe.fit(X,y)
	s8 = rfe.ranking_
	s8 = map(lambda x:round((len(s8)+1-x)/float(len(s8)),4),s8)
	s8 = np.array(s8)
	# 总分
	s = s1+s2+s3+s4+s5+s6+s7+s8
	s_sum = pd.Series(s,index=X.columns)
	s_sum.sort_values(ascending=False,inplace=True)
	return s_sum.index

In [83]:
feature_index = feature_sort(X_train,y_train)
sorted_feature = list(feature_index.values)
sorted_feature_copy = list(sorted_feature)
selected_feature = []
feature,score,clf = feature_selection(X_train,y_train,sorted_feature,selected_feature)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedLasso is deprecated; The class RandomizedLasso is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


In [73]:
sorted_feature_copy

['psd_cha_25',
 'psd_mean_25',
 'psd_var_3',
 'psd_cha_1',
 'psd_var_1',
 'psd_var_2',
 'psd_mean_27',
 'psd_cha_15',
 'psd_cha_26',
 'psd_var_4',
 'psd_mean_29',
 'psd_var_19',
 'psd_var_25',
 'psd_var_7',
 'psd_mean_26',
 'psd_var_8',
 'psd_var_29',
 'psd_var_17',
 'psd_cha_14',
 'psd_var_24',
 'psd_mean_3',
 'psd_var_18',
 'psd_cha_18',
 'psd_mean_4',
 'psd_var_5',
 'psd_var_9',
 'psd_mean_5',
 'psd_var_10',
 'psd_var_21',
 'psd_var_28',
 'psd_cha_0',
 'psd_var_22',
 'psd_var_6',
 'psd_var_13',
 'psd_mean_28',
 'psd_var_26',
 'psd_mean_2',
 'psd_var_16',
 'psd_mean_6',
 'psd_mean_7',
 'psd_mean_8',
 'psd_mean_9',
 'psd_cha_3',
 'psd_cha_6',
 'psd_var_20',
 'psd_var_14',
 'psd_cha_21',
 'psd_var_12',
 'psd_var_0',
 'psd_var_11',
 'psd_var_23',
 'psd_var_15',
 'psd_var_27',
 'psd_cha_16',
 'psd_mean_22',
 'psd_mean_21',
 'psd_cha_2',
 'psd_cha_20',
 'psd_mean_20',
 'psd_mean_24',
 'psd_mean_1',
 'psd_mean_19',
 'psd_mean_23',
 'psd_cha_24',
 'psd_mean_12',
 'psd_mean_18',
 'psd_mean_1

In [84]:
feature

['psd_cha_25', 'psd_mean_25', 'psd_var_3', 'psd_mean_27']

In [85]:
score

0.98333333333333328

In [88]:
clf.predict(X_test[feature])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [76]:
y_test.values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [49]:
# SVM 模型
from sklearn.model_selection import GridSearchCV
parameters={'kernel':('rbf','linear','poly','sigmoid'),'C':[0.01,0.1,1,5,10]}
svr = SVC()
clf = GridSearchCV(svr,parameters,scoring='roc_auc',cv=10)
clf.fit(X_train,y_train)
print(clf.best_estimator_)
print clf.best_score_ 

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.927083333333


In [78]:
from sklearn.model_selection import GridSearchCV
parameters={'kernel':('rbf','linear','poly','sigmoid'),'C':[0.0000001,0.001,0.01,0.1,1,5,10,20]}
svr = SVC()
clf = GridSearchCV(svr,parameters,scoring='roc_auc',cv=10)
clf.fit(X_train_s,y_train)
print(clf.best_estimator_)
print clf.best_score_ 

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.856770833333


In [18]:
# LR  模型
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
parameters={'C':[0.001,0.01,0.1,1,5,10,20]}
lr = LogisticRegression()
clf = GridSearchCV(lr,parameters,scoring='roc_auc',cv=10)
clf.fit(X_train,y_train)
print(clf.best_estimator_)
print clf.best_score_ 

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
1.0


In [30]:
# LR  模型
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
parameters={'C':[0.0000001,0.001,0.01,0.1,0.5,1,5,10,20]}
lr = LogisticRegression()
clf = GridSearchCV(lr,parameters,scoring='accuracy',cv=10)
clf.fit(X_train_s,y_train)
print(clf.best_estimator_)
print clf.best_score_

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.95


In [31]:
clf_train = clf.best_estimator_
#clf_train.score(X_test_s,y_test)

In [32]:
clf_train.predict(X_test_s)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
y_test.values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [132]:
#predict
#path_predict = './0908/NO/GL/2455/NO_SA_2455_20_5000_00'
#path_predict = './0908/TC/GL/140/2425/TC_SA_2425_140_20_5000_00'
#path_predict = './0711/N/SA/SA2405/SA00'
path_predict = './0711/Y/gong/SA/SA2415-80/SA00'
n = 400
x_predict = get_real_data(path_predict)
#x_predict_samp = x_predict[range(0,5000,5)]
#x = x_predict.values
y_predict = clf.predict(x_predict)
print y_predict

[1 0 0 1 1]


In [39]:
x_predict

array([ 1.,  1.,  1., ..., -1., -1., -1.])

In [48]:
len(x_predict)

3000

In [19]:
len(x_predict)

10001

In [57]:
path_predict = './0908/NO/GL/2475/NO_SA_2475_20_5000_00'
#path_predict = './0908/TC/GL/110/2475/TC_SA_2475_110_20_5000_00'
#path_predict = './0711/N/SA/SA2415/SA00'
#path_predict = './0711/Y/gong/SA/SA2475-80/SA00'
n = 400
x_predict = get_real_data(path_predict,n)
print x_predict.shape

(10001,)


In [41]:
a = x_predict.values

In [43]:
a[index_sel].shape

(1000,)